Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the related LOs requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>CS152 Assignment 2: The DPLL Algorithm</h1>

<h1>Question 1</h1>

Define your <code>Literal</code> class below.  Ensure that you define a <code>name</code> and <code>sign</code> attribute as required in the assignment description.  In addition, include any other attributes and member functions as needed. You will need to overload <code>__neg__()</code> to correctly handle negated literals.  You may need to overload <code>__eq__()</code> and <code>__hash__()</code> also.

In [1]:
class Literal:
    
    """Class Literal: Provides a structure for representing the literals when 
    performing a DPLL algorithm."""
    
    def __init__(self, name, sign=True):
        self.name = name #name of the literal 
        self.sign = sign #sign property 
        
    def __neg__(self):
        """Handles negated literals."""
        
        return Literal(self.name, not self.sign)
    
    def __repr__(self):
        """For better representation of the literals and for easier debugging."""
        sign = '' if self.sign else '-'
        return sign + self.name

    def __eq__(self, other): 
        """Compares objects and checks for equality."""
        return self.name == other.name
    
    def __str__(self): 
        """Printable string representation of the object."""
        return "-" + self.name if not self.sign else self.name
    
    def __hash__(self): 
        """Returns the hash value of an object."""
        return hash(self.name)
    

<h1>Question 2</h1>
Implement DPLL by filling in the stubs below.  Note that the various heuristics are not required to be implemented for basic functionality, but a template has been provided for you to do so if you attempt the extension questions

DPLL is an algorithm used for checking propositional logic formulae given in the conjunctive normal form (CNF) for satisfiability (i.e., whether there exists a model where all of the clauses in the knowledge base (KB) are true). It takes as input a sentence in CNF (i.e., a set of clauses) and recursively enumerates all possible models in a depth-first manner. 

Below is the implementation of the DPLL algorithm. It is simplified compared to the version in the book's pseudo-code, but it still works.

Almost every step is explained in the code comments, but in short, for no heuristic version, my implementation randomly chooses a symbol from the set of unique symbols present in the KB and creates two branch models with that symbol being assigned True and False. Then, the algorithm checks whether there are the same signed symbols present in each of the clauses of the KB, and if that is true for the clause, an arbitrary `Pass` symbol is added. This branching of different symbols truth-value assignment continues unless the model turns out to have all clauses being True or the algorithm finds all symbols in the current model present in one of the clauses but with different signs. If that happens, this means that one clause is automatically False, and hence, the algorithm stops and gives False for that model. The search continues with different models, and if it happens that all clauses are True (i.e., if they contain the `Pass` symbol), it automatically returns True, the model itself, and the symbols chosen. 

In [2]:
from copy import * #library needed for implementing the DPLL function

def DPLL_Satisfiable(KB, heuristic_level=0):
    
    """ satisfiable, model = DPLLSatisfiable(KB)
        Takes in a KB and returns whether the KB is satisfiable, and the model that makes it so.
        
        Input:
            -KB: A knowledge base of clauses (CNF) consisting of a list of sets of literals.  A KB might look like
            [{A,B},{-A,C,D}].
            -heuristic_level: An integer that will be passed in to specify which heuristics to implement 
            (only for the extension activities).
        
        Output:
            -satisfiable: Returns True if the KB is satisfiable, or False otherwise.
            -model: A dictionary that assigns a truth value to each literal for the model that satisfies KB.
            For example, a model might look like {A: True, B: False}. If there are free variables, it doesn't show them 
            in  the model in the first place, so in the above example, we assume that C, D, E, and F are free.
            -model_keys: Returns a symbol trace for the given method. """
    
    #a list for saving the trace 
    global model_keys
    model_keys = []

    #this Pass symbol is needed to see whether any of the symbols for the given model are true, which will then help
    #identify that the whole clause is true and based on the presence of Pass, we could see whether all clauses are true
    #given some specific model since we are dealing with CNF and disjunctions within them
    Pass = Literal('Pass')
    
    #creating a set of literals 
    #adding all the KB literals into the set (only unique ones without repetition)
    symbols = set() 
    for clause in KB:
        for literal in clause:
            symbols.add(literal)

    def DPLL(clauses, new_clauses, symbols, model, model_keys):
        
        #iterating over the literals in each of the clause in the KB
        for cl, clause in enumerate(clauses, 0):
            count = 0 
            #only run for the clauses which are not true yet, because it is enough to find one True literal to make the 
            #disjunction True
            if not Pass in clause:
                for literal in clause:
                    #if the literal we are setting to some boolean value in the model is in the clause, then keep 
                    #count of how many literals match
                    if literal.name in model:
                        count += 1
                        #if the signs of these matched literals are the same, then it means that the whole clause is True
                        #since we are dealing with disjunctions and if one literal out of the clause is True, then the whole 
                        #clause is True, which is why we add a P symbol (True Clause) to help us know which clauses are True
                        if literal.sign == model[literal.name]:
                            clause.add(Pass)
                            #this is needed for the implementation of the heuristics, where the clauses are updated
                            new_clauses[cl].add(Pass)
                            break  
                        #for the heuristic implementation, we discard the literals of the wrong sign since we know they are false 
                        #for the model; if all literals are discarded, the model is False since the clause is False
                        elif literal.sign != model[literal.name]:
                            if literal in new_clauses[cl]:
                                new_clauses[cl].remove(literal)
                                if new_clauses[cl]==set():
                                    return False, {}, []
                        #if all of the literals in the clauses are already in the model but their signs don't match, 
                        #then it means that all literals in the clause are False and the model is False
                            if count == len(clause):
                                return False, {}, []
        
        #if all clauses are true, then this model is True and KB satisfiable, since all of the clauses have at least
        #one literal which converged to True as seen by the presence of P
        if all([Pass in clause for clause in clauses]):
            change={}
            new={}
            for literals in model:
                change[literals]=Literal(literals, model[literals])
            for literal in model:
                new[change[literal]]=change[literal].sign
            return True, new, model_keys
         
        #updating the clauses
        #I couldn't just update the clauses directly because the set sizes would have changed 
        #within the loops, so I used a new list where the True clauses are deleted and the wrong sign 
        #literals are discarded from the clauses
        clauses = [clause for clause in clauses if not Pass in clause]
        new_clauses = [clause for clause in new_clauses if not Pass in clause]
        
        if heuristic_level == 0 :
            #choose an arbitrary literal from a set of unique literals
            choice = symbols.pop()
        if heuristic_level == 1:
            #choose the most common literal among the leftover clauses
            choice = degree(symbols, new_clauses)
            symbols.remove(choice)
        if heuristic_level == 2:
            #choose the pure literal; if none, choose unit clause literal; if none, choose the most common literal
            #among leftover clauses; if multiple, choose alphabetically 
            choice = pure_unit(new_clauses,symbols)
            symbols.remove(choice)
        if heuristic_level == 3:
            #choose the pure literal; if none, choose unit clause literal; if none, choose the most common literal
            #among leftover clauses; if multiple, choose the most common
            choice = pure_unit(new_clauses,symbols,common=True)
            symbols.remove(choice)
        
        #the symbol trace for the methods    
        model_keys.append(choice)
        
        #since we are working with recursion, we use deep copy to construct a new compound object (i.e., model object) 
        #and then recursively insert copies into it of the objects found in the original 
        model1, model2 = deepcopy(model), deepcopy(model)
        #making two possible models for the literal - True or False
        model1[choice.name], model2[choice.name] = True, False
                
        #recursively calling DPLL to test satisfiability 
        satisfiable, Model, model_keys1 = DPLL(deepcopy(clauses), deepcopy(new_clauses), deepcopy(symbols), model1, model_keys)
        #returning the output if the model turned out to be true
        if satisfiable:
            return satisfiable, Model, model_keys1
                
        #doing the same thing as above but for another version 
        satisfiable, Model, model_keys1 = DPLL(deepcopy(clauses), deepcopy(new_clauses), deepcopy(symbols), model2, model_keys)            
        if satisfiable:
            return satisfiable, Model, model_keys1
        
        #if we don't have any more unique unused literals and the algorithm couldn't find a model where all of 
        #the clauses are True, then it means that no such model exists and the KB is not satisfiable 
        return False, {}, []
    
    return DPLL(KB, list(KB), symbols, {}, model_keys)


**Note:** after implementing the DPLL algorithm and already passing all extension tests, I realized that I could have made an easier version of this code without the inclusion of the `Pass` symbol, although it was an easy way to understand how DPLL works. Similarly to how the new_clauses local variable updates, I could have passed as arguments for the clauses in the DPLL function the updated clauses already without the True clauses nor the discarded wrong sign literals. This would have shortened my code, but the result would have been the same. 

<h1>Question 3</h1>

Implement your KB from Russell & Norvig in CNF by filling in the sets inside the list <code>KB</code> below.  Ensure that your KB is in a list of set format as stated in the assignment instructions.

![pic](https://scontent.ftse2-1.fna.fbcdn.net/v/t1.15752-9/92442128_2637654039676658_501553876961329152_n.png?_nc_cat=106&_nc_sid=b96e70&_nc_eui2=AeHIGMJHDN3W48Ls_cgxCvgjjxDSt0dFdMZaxB_um5spgSyxjtqWngkRUIEZPYuWsNmDdYJGb_mNd6DSDmv4SCtFcyQTLymGzew1A8sXLAKSKg&_nc_ohc=N87QRYpjLKUAX8E9fxm&_nc_ht=scontent.ftse2-1.fna&oh=9f3d696fb717fe28278c8ffa31e4631d&oe=5EAC2F23&dl=1)

Using the basic rules and transformations, we can transform the above sentences into CNF:

$$S_1: ((\neg A) \lor (B \lor E)) \land ((\neg B \land \neg E) \lor A)=(\neg A) \lor B \lor E) \land (\neg B \lor A) \land (\neg E \lor A)$$

$$S_2: \neg E \lor D$$

$$S_3: \neg C \lor \neg F \lor \neg B$$

$$S_4: \neg E \lor B$$

$$S_5: \neg B \lor F$$

$$S_6: \neg B \lor C$$

The above sentences are all in CNF, so we can add them into the KB as such.

In [3]:
#defining all literals
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

#defining KB
KB = [{-A, B, E},{-B, A},{-E, A},
      {-E, D},
      {-C, -F, -B},
      {-E, B},
      {-B, F},
      {-B, C}]

#seeing whether the KB is satisfiable and has solutions
satisfiable, model, keys = DPLL_Satisfiable(KB, 0)
model

{D: True, C: True, -B: False, -E: False, -A: False}

<h1>Basic Functionality Tests</h1>

All of the tests in this section must be passed for the code to be considered fully functional.  Other unseen test cases will be used after submission to further verify functionality

In [4]:
# This section will test the correct definition of the literal class


# Test the name attribute works correctly
assert(A.name == 'A')

# Test that negation works correctly
assert(not (-C).sign)

In [5]:
# This section will test that the KB has been correctly converted to CNF by performing 
# tests over all possible models
import itertools
symbols = {A,B,C,D,E,F}
symbol_list = [A,B,C,D,E,F]

def evaluate_russell_norvig_KB(model):
    # Manually evaluate the KB using the model
    s = list()
    s.append(model[A] == (model[B] or model[E]))
    s.append(model[E] <= model[D])
    s.append((model[C] and model[F]) <= (not model[B]))
    s.append(model[E] <= model[B])
    s.append(model[B] <= model[F])
    s.append(model[B] <= model[C])
    return all(s)

def evaluate_KB(KB, model):
    model_true = True
    model_name_dict = {l.name: model[l] for l in model}
    for clause in KB:
        evaluation = {l.sign == model_name_dict[l.name] for l in clause if l.name in model_name_dict}
        model_true = model_true and any(evaluation)
    return model_true

all_models = list(itertools.product([False, True], repeat=6))
for i in range(0, len(all_models)):
    # Test all truth values
    test_model = {symbol_list[j]: all_models[i][j] for j in range(0,6)}
    
    # Test the model
    assert(evaluate_russell_norvig_KB(test_model) == evaluate_KB(KB, test_model))

In [6]:
# This section will test the basic working of DPLL
# Satisfiable model
test_KB = [{A, C},{-A, C}, {B, -C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(evaluate_KB(test_KB, model))
assert(t)

# Unsatisfiable model
test_KB = [{A, C},{-A, C}, {-C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(not t)

In [7]:
# This will test DPLL on the KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,0)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model)) 

{D: True, C: True, -B: False, -E: False, -A: False}


<h1>Extensions</h1>

1. Implement the degree heuristic for choosing symbols.  If <code>heuristic_level=1</code>, then the degree heuristic should be used to select which symbol to assign.
2. Implement the pure symbol and unit clause heuristics.  If <code>heuristic_level=2</code>, then these heuristics should be used to find select first a pure symbol, and if one is not found, then a unit clause.  If neither pure symbols nor unit clauses are found, then the degree heuristic should be used to select a symbol.  If there are multiple pure symbols or unit clauses found, then use alphabetical order to select amongst them.
3. Modify the pure symbol heuristic to choose the pure symbol that occurs in the most number of clauses.  This should be activated if <code>heuristic_level=3</code>.

**Degree Heuristic**:

Below is the implementation of the degree heuristic, which basically finds the most common literal based on the list of available literals as well as the leftover clauses. 

In [8]:
#the degree heuristic implemenetation

def degree(symbols,clauses):
    
    """ This function returns the most common literal from the list of available literals 
    and the leftover clauses.
    
    Input:
        -symbols: the list of symbols which were not already chosen (unique).
        -clauses: the clauses list which is still not resolved given the model.
        
    Output:
        -degree: the most common literal. """
    
    #finds out how many times a literal has been present in the clauses
    num_of_symbols = {}
    for symbol in symbols:
        for clause in clauses:
            for literal in clause:
                if literal == symbol:
                    if literal in num_of_symbols:
                        num_of_symbols[symbol] += 1
                    else:
                        num_of_symbols[symbol] = 1
                        
#     after I passed the first extension test, I realized that it was easier to
#     sort the literals alphabetically if I just used their names as in the 2-3 extensions
#     so below is an alternative code although it just gives the letter instead of a 
#     literal, which could be further fixed
                    
#     sorted_list = sorted(num_of_symbols, key=lambda item: num_of_symbols[item])
#     return sorted(sorted_list, key=name_alph)
    
    #we want to return the most common literal and if there are multiple, then the 
    #literal alphabetically, which is why we sort first based on the value of the dictionary 
    #(i.e., number of appearances) and then alphabetically - the code above is an easier alternative
    
    change={}
    new={}
    for literals in num_of_symbols:
        change[literals]=literals.name
    for literal in num_of_symbols:
        new[change[literal]]=num_of_symbols[literal]
                
    sorted_symbols = sorted(new, key=lambda x: (-new[x], x))
    for i in range(len(sorted_symbols)):
        sorted_symbols[i]=Literal(sorted_symbols[i])
    
    degree = [sorted_symbols[i] for i in range(len(sorted_symbols))][0]
    return degree

**Pure symbol + Unit clause**:

Below is the implementation of the pure symbol and unit clause heuristics. A pure symbol is a symbol which appears with a consistent sign throughout the whole leftover KB. At the same time, a unit clause is a clause that consists of a single literal. For `heuristic_level=2` and `heuristic_level=3`, if there are multiple pure symbols or unit clauses, then it uses alphabetical order or commonness to choose a symbol, respectively. When there are no pure symbols or unit clauses, a symbol is chosen according to the degree heuristic.
    

In [9]:
def pure_unit(clauses,symbols,common=False):
    
    def name_alph(literal):
        """This function returns the name of the literal for sorting alphabetically."""
        return literal.name   
    
    """ This function implements the pure symbol and unit clause heuristics. If there are
    multiple pure symbols or unit clauses, then it uses alphabetical order or commonness to choose a symbol.
    When there are no pure symbols or unit clauses, a symbol is chosen according to the degree heuristic.
    
    Input:
        -symbols: the list of symbols which were not already chosen (unique)
        -clauses: the clauses list which is still not resolved given the model
        
    Output: suitable literal. """  
        
    pure_unit_symbols = set()
    
    #getting all of the literals in the clauses into one big list
    all_symbols=[]
    for clause in clauses:
        for literal in clause:
            all_symbols.append(literal)
      
    #for all available literals, we scroll through the all_symbols to collect all instances of that literal;
    #if the signs of those literals in the list are all the same, then we found a pure symbol - if not, then move further
    for symbol in symbols:
        signs = []
        while symbol in all_symbols:
            signs.append(all_symbols[all_symbols.index(symbol)])
            all_symbols.remove(symbol)
        if (all(x.sign==True for x in signs) or all(x.sign==False for x in signs)) and not len(signs)==0:
            pure_unit_symbols.add(symbol)
            
    #for heuristic2: if we have multiple pure symbols, sort them alphabetically and choose the first one
    #for heuristic3: if we have multiple pure symbols, choose the most common one
    if len(pure_unit_symbols) > 0:
        if len(pure_unit_symbols)==1:
            return pure_unit_symbols.pop()
        if common:
            return degree(pure_unit_symbols, clauses)
        return sorted(pure_unit_symbols, key=name_alph)[0]

    #if we don't have any pure symbols, go for the unit clauses;
    #since we were updating the clauses and discarding the wrong sign instances as well as discarding clauses 
    #which are already true, we will simply choose the clauses of length 1
    if len(pure_unit_symbols) == 0:
        for clause in clauses:
            if len(clause) == 1:
                pure_unit_symbols.add(clause.pop())
    #for heuristic2: if we have multiple unit clauses, sort them alphabetically and choose the first one
    #for heuristic3: if we have multiple unit clauses, choose the most common one
    if len(pure_unit_symbols) > 0:
        if common:
            return degree(pure_unit_symbols, clauses)
        return sorted(pure_unit_symbols, key=name_alph)[0]

    #if no pure symbols or unit clauses, go for the degree heuristic
    else:
        return degree(symbols, clauses)

<h1>Extension Tests</h1>

This section will test that the degree heuristic, pure symbol and unit clause heuristics are correctly implemented.  Note that in your code, the <code>heuristic_level</code> needs to correctly activate the heuristic being tested, and the <code>symbol_list</code> needs to be correctly generated 

In [10]:
# Degree Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,1)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))  

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] in [['B','F', 'E', 'C', 'C', 'E', 'A'], ['B', 'A', 'C', 'F', 'E', 'A']])

symbol_trace

{-B: False, -E: False, -A: False}


[B, A, C, F, E, A]

**Note:** I came to OH and it turned out the answer key was different, so I changed it - the actual locked cell is in the bottom of the assignment. 

In [11]:
# Pure Symbol & Unit Clause Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,2)
print(model)


# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] == ['D', 'B', 'A', 'C', 'F', 'E', 'A'])

{D: True, -B: False, -E: False, -A: False}


In [12]:
# Pure Symbol & Unit Clause Heuristic Test, choosing the most-frequently used pure symbol, for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,3)
print(model)
print(symbol_trace)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] == ['D', 'B', 'A', 'C', 'F', 'E', 'A'])

{D: True, -B: False, -E: False, -A: False}
[D, B, A, C, -F, E, -A]


In [ ]:
# Pure Symbol & Unit Clause Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,2)
print(model)


# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] == ['D', 'B', 'E', 'A', 'C', 'F', 'E', 'A'])